# Advent of Code 2020 in APL

## Day 1

In [219]:
⍝ Read file linewise, parse every line into a number
input1 ← ⍎¨⊃⎕NGET 'day1.txt' 1

In [3]:
day1_1 ← {×/⍵[⊃⍸2020⍷⍵∘.+⍵]} ⍝ ←→ {×/⍵⌷⍨⊂⊃⍸2020=∘.+⍨⍵}
day1_1 input1

436404

In [4]:
day1_2 ← {×/⍵[⊃⍸2020⍷⍵(∘.+⍣2)⍵]} ⍝ ←→ {×/⍵⌷⍨⊂⊃⍸2020=(∘.+⍣2)⍨⍵}
day1_2 input1

274879808

## Day 2

In [5]:
input2 ← ⊃⎕NGET 'day2.txt' 1
⍝ '1-2 l: phrase' ~> (1 2 l phrase)
parsed2 ← {' '(≠⊆⊢)('-|:' ⎕R ' ')⍵}¨ input2

In [6]:
⍝ Count occurrences of ⍵[3] in ⍵[4]:
occ ← {+/⊃⍵[3]=⍵[4]}
⍝ Check if ⍵[1] ≤ ⍺ ≤ ⍵[2]
between ← {((⍎⊃⍵[1])≤⍺)∧((⍎⊃⍵[2])≥⍺)}
⍝ Count correct passphrases:
day2_1 ← {+/(occ between ⊣)¨⍵}
day2_1 parsed2

434

In [7]:
day2_2 ← {≠/ (⊃⍵[3]) = (⊃⍵[4])[⍎¨⍵[1 2]]} ⍝ Check if the phrase is correct
+/day2_2¨ parsed2

509

## Day 3

In [8]:
input3 ← ⎕NGET 'day3.txt' 1
parsed3 ← ((≢,(≢⊃))⍴∊)⊃input3 ⍝ Parse input3 into a matrix of correct dimensions

In [9]:
day3_1 ← {⊃+⌿ (3×¯1+⍳≢⍵) ⌽ '#'=⍵}
day3_1 parsed3

184

In [10]:
⍝ The logic is mostly the same, but with variable sloping
day3_2 ← {⊃+⌿ (⍺×¯1+⍳≢⍵) ⌽ '#'=⍵}
⍝ Special case ⍺=÷n: take every n-th line and shift the rows 1-wise 
preproc ← {((≢⍵)⍴1,((⍺-1)⍴0))⌿⍵}
⍝ Format number in non-exponential notation
'I10' ⎕FMT (1 day3_2 2 preproc parsed3)×{1 3 5 7 ×.day3_2 ⍵} ⊂parsed3

2431272960

## Day 4

For problem one, if input is the file as one string, we want something like the following (in a Rusty syntax):
```rust
let requiredFields = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"];
let solution = input : String
    // Parsing part:
    .splitAt(emptyLine) : [&String]
    .map(|xs| xs.joinWith(' ')) : [&String]
    .map(words) : [[&String]]
    .map∘map(|xs| xs.splitAt(':')) : [[(String, String)]]
    // Logic part:
    .map∘map(first) : [[String]]
    .map(|xs| isSubset(requiredFields, xs)) : [Bool]
    .count(True);
```

In [11]:
input4 ← ⊃⎕NGET 'day4.txt' 0
nl ← (⎕UCS 10)
⍝ transform string into each line containing one key:value pair and a '-' between records
normalize ← {('^$'⎕R'-') (' ' ⎕R '\n') ⍵}
splitAt ← {⍺(≠⊆⊢)⍵}
⍝ Like splitAt, but checks for ∊ instead of =
⍝ Must start with a divider, else the first partition element is lost. :(
partitionAt ← {1↓¨⍺(∊¨⊂⊢)⍵}
⍝ process key:value pairs
entry ← {':' splitAt¨ ⍵}
parsed4 ← entry¨ '-' partitionAt nl splitAt normalize nl, input4

In [153]:
day4_1 ← {7 = ≢'byr' 'iyr' 'eyr' 'hgt' 'hcl' 'ecl' 'pid' ∩ ⊃¨⍵}
+/ day4_1¨ parsed4

242

For problem two, dictionaries would be really helpful … Well it works, somehow.

In [211]:
⍝ Check all conditions for every field
⍝ Multiline definitions seem not to work. :(
chbyr ← {(⍎⊃⍵) between '1920' '2002'}
checl ← {(⊂⍵) ∊ 'amb' 'blu' 'brn' 'gry' 'grn' 'hzl' 'oth'}
cheyr ← {(⍎⊃⍵) between '2020' '2030'}
chhcl ← {'.'∊('^#[0-9a-f]{6}$' ⎕R'.') ⊃⍵} ⍝ Terrible hack, but that’s how it is
chhgt ← {(≢'^(((59|6[0-9]|7[0-6])in)|(1([5-8]\d|9[0-3])cm))$' ⎕S 1) ⍵}
chiyr ← {(⍎⊃⍵) between '2010' '2020'}
chpid ← {9=+/('^\d{9}$' ⎕S 1) ⍵}
checkall←{∧/(chbyr ⍵[1]) (checl ⊃⍵[2]) (cheyr ⍵[3]) (chhcl ⍵[4]) (chhgt ⍵[5]) (chiyr ⍵[6]) (chpid ⍵[7])}

reduceFields ← {7=≢⍵: (2⊃¨⍵)[⍋⊃¨⍵] ⋄ 8=≢⍵: (1 0, 6⍴1) / (2⊃¨⍵)[⍋⊃¨⍵]}
+/ (checkall∘reduceFields)¨ {(day4_1¨ ⍵)/⍵} parsed4

186

## Day 5

For problem one, there are three steps (not necessarily in that order):
1. translate a code into a (row, seat) number pair
2. sort all entries/take the maximum
3. calculate `seatId ← seat + 8 × row`

The most resource efficient would probably be 2–1–3, because then only one code has to be translated, but it should not make too much of a difference, since all three functions have to be implementet no matter what. Translation works as follows:

```
         F           B          F           B          B          F          F
(0, 127) → (0, 63)   → (32, 63) → (32, 47)  → (40, 47) → (44, 47) → (44, 45) → (44, 44)
(0, 2*7) + (0, -2*6) + (2*5, 0) + (0, -2*4) + ...
```

In [253]:
input5 ← ⊃⎕NGET 'day5.txt' 1

In [289]:
translate ← +/{(2*¯1+(⌽⍳7),⌽⍳3)×⍺=⍵} ⍝ Count using B for rows, R for seats
seats ← ('B'∘translate,'R'∘translate)¨ input5
day5_1 ← {+/8 1×⍵}
⌈/day5_1¨ seats

838

In [324]:
sortedSeats ← {⍵[⍒⍵]} day5_1¨ seats
day5_2 ← {¯1+⍵[⍸1≠-⌿¯1↓[2](2, ≢⍵)⍴⍵, 1⌽⍵]} ⍝ subtract neighbors, find difference greater 1
day5_2 sortedSeats

714

## Day 6

In [357]:
input6 ← ⊃⎕NGET 'day6.txt' 0
parsed6 ← '-' partitionAt normalize nl, input6 ⍝ Reuse from day 4

In [360]:
day6_1 ← {¯1+≢∪⍵}
+/ day6_1¨ parsed6

6161

In [391]:
day6_2 ← {≢⊃∩/nl(≠⊆⊢)⍵}
+/ day6_2¨ parsed6

2971

## Day 7

In [625]:
input7 ← ⊃⎕NGET 'day7.txt' 1
parsed7 ← {(1 1 0 0, (¯4+≢⍵)⍴1 1 1 0)⊆⍵}¨ ' ' (≠⊆⊢)¨ input7

For problem one, consider a directed graph (more precise a tree), whith an edge a→b iff a∊b. Calculate the adjacency matrix A. The n-th power of A gives the n-deeply nested containment, the transitive closure is the fixed point of $A \mapsto \operatorname{sgn}(A + A \times A)$.

Thus, the problem is divided into two parts:
1. Find the transitive closure T of the adjacency matrix for the baggage graph
2. Find the index of 'shiny gold' bags and +/ the the corresponding row of T

In [626]:
parsed7_1 ← {⍵[1], 1↓¨1↓⍵}¨ parsed7
eye ← {⍵ ⍵⍴1, ⍵⍴0} ⍝ unit matrix of shape ⍵ ⍵
traclo ← {⍵∨⍵∨.∧⍵}⍣≡ ⍝ transitive closure
⍝ For every bag, in which bags is it contained? (I. e. is there an edge?)
day7_1 ←{+/ (traclo (-eye ≢⍵)+(⊂¨⊃¨⍵) ∘.∊ ⍵)[((⊃¨⍵) ⍳ ⊂'shiny' 'gold');]}
day7_1 parsed7_1

257

For problem one, we have to go the other way around. An edge a→b means that a contains b; also we have a multigraph now, meaning that each entry in A contains the number of edges given by the input.

In [ ]:
foos ← (('foo' (4 'bar')) ('bar' ('no' 'more')) ('ham' ('no' 'more')) ('spam' (1 'quux') (2 'ham')) ('quux' (1 'foo') (3 'baz')) ('baz' ('no' 'more')))
⍝ Transform empty bags into the right format;
parsed7_2 ← {'no'≡⊃2⊃⍵:⊂⊃⍵ ⋄ ⍵}¨ parsed7
len ← ≢ parsed7_2
⍝ number of contained bags:
numbers7 ← {⍎¨¨(1⊃¨¨1↓¨⍵),¨'0'} parsed7_2
⍝ names of the bags contained in each bag, but catenated à la 'shiny' 'gold' → 'shinygold'
names7 ← {∊¨¨1↓¨¨1↓¨⍵} parsed7_2

⍝ [i; j] = index or ≢parsed7_2, depending on if a bag contains a bag
indices7 ← {⊃¨⍵,¨≢⍵}(⍸¨names7 ∘.(≡¨) (⊂¨∊¨⊃¨parsed7_2)) 
⍝ [i; j] = (x y) such that (numbers7[x])[y] = adjacency[x;y]
indices7_2 ← (⍉(len len)⍴ ⍳⍴ numbers7) ,¨ indices7 ⌊ ⍉(len len)⍴ ⍴¨numbers7
⍝ The adjacency matrix (don’t ask me why):
adj ← {⊃(numbers7[⍵1])}¨ indices7_2
goldIndex ← ⍸(⊂'shinygold')⍷∊¨⊃¨parsed7_2
⍝ 9 is a magic number right now; it should be n given by when adj*n ≡ 0 , but how?
⍝ with adj(+.×⍣≡)adj we get the fixed point; we want to scan until that is reached.
+/⊃+/goldIndex ⌷¨ ((+.×⍣≡)\ 9⍴ ⊂⊃¨adj)

## Day 8

In [699]:
input8 ← ⊃⎕NGET 'day8.txt' 1
parsed8 ← ' ' (≠⊆⊢)¨ input8

For problem one, first go through the instructions and create an array of the lines to be executed in the right order; loop detection also happens here. Look up the `(cmd, val)` pairs at these lines and filter out all `nop`s and `jmp`s. The sum of the result is the solution.

Multiline function definitions seem to have to be in their own cell. Thus, there are more than three cells for this (and following) days.

In [696]:
]dinput
cmds8←{ ⍝ List all commands that are actually executed, until repeat
    ⍺ ← 1 ⍝ Default value for monadic execution
    last ← ¯1↑⍺
    last∊¯1↓⍺: ¯1↓⍺ ⍝ Loop/duplicate detection
    last>(≢⍵): 0, ⍺ ⍝ PC > #cmds, program terminates; added for problem two
    cmd ← ⍵[last]
    'jmp'≡⊃⊃cmd: (⍺, last+⍎2⊃⊃cmd)∇⍵ ⍝ Change PC for jmp
    (⍺, last+1)∇⍵
}

In [628]:
⍝ ⍵ the to-be-executed cmds; filter out all `jmp`s and `nop`s, only retain `acc`s.
day8_1 ← {+/⍎¨2⊃¨({'acc'∘≡⊃⍵}¨ ⍵)/⍵}
day8_1 parsed8[cmds8 parsed8]

1797

For problem two, we need to generate every possible variation of the program by flipping one `nop`/`jmp` at a time. Then, every program is tested for termination, and only the single terminating program is filtered out; this is then executed.

In [459]:
]dinput
flipCmds ← { ⍝ Flip command at position ⍺
    cmd ← ⊃⍵[⍺]
    'acc'≡⊃cmd: ⍬ ⍝ Return nothing for 'acc', since 'acc' cannot be changed
    ('nop' '+0')≡cmd: ⍬ ⍝ jmp +0 is illegal, return nothing
    cmd ← ⊂('nop' 'jmp')[('jmp' 'nop')⍳⊂⊃cmd], ⊂2⊃cmd 
    ((⍺-1)↑⍵), cmd, ⍺↓⍵ ⍝ jmp → nop always works
}

In [700]:
⍝ Generate all possible programs, remove the trivial changes,
⍝ choose the terminating one (with the prepended 0; which is then dropped).
⍝ This could potentially be prettier if instead of the zero, it is looked for
⍝ an Index that is out of bounds.
correct ← 1↓ ¯1↓⊃{(0∊¨⍵)/⍵} cmds8¨ {(~(0∊⍴)¨⍵)/⍵} {(⍳≢⍵)∘.flipCmds⊂⍵} parsed8
day8_1 parsed8[correct]

1036

## Day 9

In [169]:
input9 ← ⍎¨⊃⎕NGET 'day9.txt' 1

For problem one, get the `n` numbers before `j` by rotating the input by `j`. We ignore the first `n` rows, since they are the preamble, and then search the first `n` entries per to sum to the `n+1`-th entry, similar to day1. Keep in mind that the preamble is treated specially.

In [170]:
⍝ Vector of the previous ⍺ entries for each entry of ⍵, beginning with index ⍺+1.
⍝ The last set of ⍺ entries doesn’t have to be checked.
previous ← {¯1↓⍺ ,/ ⍵}
⍝ Variation on day1_1 with sum-checking for every entry.
isSum ← {(⍺↓⍵)∊¨{⍵∘.+⍵}¨ ⍺ previous ⍵}
⍝ Get the correct entry; ⍺ and ⍵ as usual.
day9_1 ← {⍵[⍸(⍺⍴0),~⍺isSum⍵]}
25 day9_1 input9

88311122

For problem two, generate all contiguous sets of numbers, sum them up and check for equality with the goal number. If equal, sort the set and add the smallest and largest of the numbers.

In [181]:
⍝ All possible contiguous subsets:
contigs ← {⊃∪/(1↓⍳≢⍵),/¨⊂⍵}
goal ← 25 day9_1 input9
⍝ Due to workspace limitations, we need to drop the entries after goal; this is valid (at leat in our case), since
⍝ +/goal ≥ 564↓ input9 ←→ 0.
⍝ Find the correct contiguous set; sort it and add the first and last element:
day9_2 ← {{+/⍵[(⊃,(¯1∘↑))⍒⍵]} ⊃{⍵[⍸goal=+/¨⍵]}contigs (⍵⍳goal)↑⍵}
day9_2 input9

13549369

## Day 10

In [415]:
input10 ← ⍎¨⊃⎕NGET 'day10.txt' 1

In [397]:
⍝ Add a 1 for the outlet and a 3 for the device
day10_1 ← {×/+/¨1,¨((3∘=),(1∘=)),-/1 0⌽¨⊂⍵[⍋⍵]}
day10_1 input10

1820

For problem 2, all adaptors with 3-jolt difference have to be in. Adaptors with a difference of 1 jolt can potentially be removed, according to the following removal scheme (dependent on the number of contiguous 1’s):
* 3 1 3 → the 1 cannot be removed, ergo 1 possibility
* 3 1 1 3 → only the first 1 can be removed, ergo 2=1+1 possibilities
* 3 1 1 1 3 → can remove none, the 1st, the 2nd or both, ergo 4=1+2+1 possibilities
* 3 1 1 1 1 3 → can remove none, 1st, 2nd, 3rd, or any two of them, ergo 7=1+3+3 possibilities
* 3 1 1 1 1 1 3 → 13=1+4+6+2

These are (as I found out afterwards) the [Tribonacci numbers](https://oeis.org/A000073): $\operatorname{Possibilities}(n) = T(n+2)$, where $n$ is the number of 1’s and $T(0)=T(1)=0, T(2)=1, T(n+3)=T(n+2)+T(n+1)+T(n)$.

In [416]:
⍝ Partition into sets of contiguous 1’s, count their length and transform according to notes above.
combinations ← 1 2 4 7 13
day10_2 ← {×/combinations[≢¨3(≠⊆⊢)1,¯1↓⊃-/1 0⌽¨⊂⍵[⍋⍵]]}
'I13' ⎕FMT day10_2 input10

3454189699072

## Day 11

In [1183]:
input11 ← ↑⊃⎕NGET 'test_input.txt' 1

Problem one is more or less Game of Life, only with different numbers and the constraint that only some cells can be inhabited.

In [1312]:
⍝ Add padding as a border, so that wraparound won’t happen.
seats ← ¯1+'.L'⍳input11
bSeats ← (0⍪0,seats,0)⍪0
occupied ← (⍴seats)⍴0
bOccupied ← (⍴bSeats)⍴0
⍝ a seat will be occupied, iff it has 0 neighbors or it is occupied and has 5 or less neighbors
step ← {bSeats ∧ ⊃(0∘=∨((⊂⍵)∧≤∘4))+/,¯1 0 1∘.⊖¯1 0 1∘.⌽⊂⍵}
+/,(step⍣≡⊢ bOccupied)

37

For problem two, cells count as neighboring if they are in a direct line of each other, with only floor spaces between them. So we need a new idea:
* Find all neighbors of every cell, if they exist – this is a 3×3 matrix (this is the difficult part); idea:
    * Generate eight vectors, concisting of all cells in each of the eight directions
    * Find the indices of the first occupied cells in each vector
    * Offset the indices by the index of the cell itself
* Sum up every such matrix and check as usual

In [1262]:
]dinput
directions←{ ⍝ for index ⍺ and seat matrix ⍵, return next seat in every direction.
    row  ← ⊂(('*'@(⊂⍺))⍵)[1⊃⍺;]
    col  ← ⊂(('*'@(⊂⍺))⍵)[;2⊃⍺]
    ⍝ I have no idea, why we need the "2+" offset, but that seems to fix it
    diag ← ⊂ ⍺ {⍵[⊃(⍴seats)+⍺;]} ⍺ {(2+(⍳2⊃⍴⍵)-2⊃⍺)⊖⍵} padMany (('*'@(⊂⍺))⍵)
    ⍝ Here, we have a "2-" offset
    anti ← ⊂ ⍺ {⍵[⊃(⍴seats)+⍺;]} ⍺ {(2-(2⊃⍺)-⍳2⊃⍴⍵)⊖⍵} padMany (('*'@(⊂⍺))⍵)
    ⍝ Split each at '*' to get the 8 directions, invert all "incoming" directions
    directions ← 1⍳¨⍨(⌽¨@1 3 5 7)⊃,/'*'(≠⊆⊢)¨ row, col, diag, anti
    ⍝directions
    ⍝ rearrange into a nice matrix
    3 3 ⍴(directions, 0)[5 3 8 1 9 2 7 4 6]
}

In [1218]:
]dinput
neighbors←{ ⍝ Calculate the indices of all "neighboring" seats for seat with index ⍵
    (⊂⍵) + (⍵ directions bSeats) × 3 3 ⍴ (¯1 ¯1)(¯1 0)(¯1 1) (0 ¯1)(0 0)(0 1) (1 ¯1)(1 0)(1 1)
}

In [1249]:
⍝ Add 0-padding on above and below the matrix
padMany ← {(-2⊃⍴⍵)⊖⍵⍪((2 1×⍴⍵)⍴0)}

⍝ Filter out seats that are out of bounds; these are not occupied
valid ← {⊃∧/~(0,1+⍴⍵)∊¨¨ ⊂neighbors⍵}

⍝ Count occupied seats neighboring ⍺, where ⍵ is the occupation matrix
count ← {+/⍵[(⊂⍬)~⍨,(valid ⍺)/¨(neighbors ⍺)]}

⍝ Will ⍺ be occupied in the next step, according to current occupation matrix ⍵?
step ← {⍵[⊃⍺;2⊃⍺] {((1=⍺)∧4≤⍵)∨0=⍵} ⍺ count ⍵}

⍝ Occupied seats after one step are seats∧(indices step occupied)

⍝(⊂¨1+⍳(⍴seats)) step¨¨ (⊂⊂occupied)
⍝(,⊂¨⍳10 10) step¨¨ (⊂⊂occupied)
⍝(1+⊂¨⍳(⍴5 5)) count¨¨ ⊂⊂occupied
(1+⊂¨2 2⍴(3 3) (4 6) (5 1) (9 9)) count¨¨ ⊂⊂occupied
⍝⊂¨1+⍳(¯1+⍴seats)
11 11 {⊂ ⍺ {⍵[⊃(⍴seats)+⍺;]} ⍺ {(2+(⍳2⊃⍴⍵)-2⊃⍺)⊖⍵} padMany (('*'@(⊂⍺))⍵)} bSeats

0 0
0 0

┌───────────────────────┐
│0 1 1 1 1 0 1 0 1 0 * 0│
└───────────────────────┘

In [1338]:
⍝(1+⊂¨2 2⍴(1 1) (1 2) (2 1) (2 2)) count¨¨ ⊂⊂seats
⍝ (1+⊂¨(1 1) (5 5) (9 9)) count¨¨ ⊂⊂occupied
⍝(⊃¨(1+⊂¨⍳(⍴seats)) {⍺{(⊂⍬)~⍨,(valid ⍺)/¨(neighbors ⍺)}¨¨⍵} ⊂⊂occupied)
neighbors ⍳2 2

RANK ERROR
directions[1] row←⊂(('*'@(⊂⍺))⍵)[1⊃⍺;]
                                  ∧


## Day 12

For problem one, transform the relative directions to absolute, then calculate N/S and W/E independently.

In [1642]:
input12 ← (⊃,(⍎1∘↓))¨ ⊃⎕NGET 'day12.txt' 1

In [1636]:
⍝ Get value for key ⍵ from assiciative list ⍺. Stolen from dfns.dyalog.org
alget←{keys vals←⍺ ⋄ (keys⍳⊂⍵)⊃vals}
⍝ Action ⍺ gets translated into ⍵⌽'NESW'
turns ← ((' ' 0) ('R' 90) ('R' 180) ('R' 270) ('L' 90) ('L' 180) ('L' 270)) (0 1 2 3 ¯1 ¯2 ¯3)
⍝ For every action, the direction the ship is facing to at the beginning of the action:
dirs ← ¯1↓+\1,(turns∘alget)¨ {({((⊃⍵)∊'LR')-(⊃⍵)∊'NESWF'}¨⍵)/⍵} input12
⍝ ⍺ the current direction, ⍵ the current action, change every 'F' into the corresponding direction
⍝ and every 'L' and 'R' into ('N' 0) – we could also use ⍬. Leave absolute movements unchanged.
transform ← {'F'=⊃⍵: (⊃⍺⌽'NESW'), 2⊃⍵ ⋄ (⊃⍵)∊'LR':'N' 0 ⋄ ⍵}

⍝ Counting
northSouth←{+/(2⊃¨⍵)×((⊃¨⍵)∊'S')-(⊃¨⍵)∊'N'} ⍝ N is negative, S is positive
eastWest←{+/(2⊃¨⍵)×((⊃¨⍵)∊'E')-(⊃¨⍵)∊'W'} ⍝ W is negative, E is positive

⍝ Combining the counts
absolute←{(northSouth⍵),eastWest⍵}
+/|absolute (dirs transform¨ input12)

1565

For problem two, step through the actions recursively.

In [1629]:
⍝ ⍺ the amount of rotation, ⍵ the position
rotL ← {ns ew ← ⍵ ⋄ ⍺=90: (-ew) ns ⋄ ⍺=180: -ns ew ⋄ ⍺=270: ew (-ns)}

In [1]:
]dinput
day12_2←{ ⍝ ⍺≡((yship xship) (ywp xwp)), ⍵ the remaining actions
    key amt ← ⊃⍵
    ship wp ← ⍺
    key∊'NESW': (ship (wp+amt×⊃(¯1 0) (0 1) (1 0) (0 ¯1)['NESW'⍳key])) ∇ (1↓⍵)
    key='L': (ship (amt rotL wp)) ∇ (1↓⍵)
    ⍝ Express rotation to the right by rotation to the left:
    key='R': (ship ((360-amt) rotL wp)) ∇ (1↓⍵) 
    key='F': ((ship+amt×wp) wp) ∇ (1↓⍵)
    ⊃⍺
}

In [1665]:
+/|((0 0) (¯1 10)) day12_2 input12

78883

## Day 13

In [6]:
input13 ← ⊃⎕NGET 'day13.txt' 1
arrival ← ⍎⊃input13
ints ← {⍎¨⍵(∊⊆⊣)⎕D}
buses ← ints 2⊃input13
⍝ Filter for the relevant buses
filter ← 'x'≠⊃¨ ','(≠⊆⊢)2⊃input13
⍝ first row are offsets, second row are bus IDs 
xbuses ← ↑(⊂filter/¯1+⍳≢filter),⊂⍎¨filter/','(≠⊆⊢)2⊃input13

For problem one, for arrival `a` and bus `b`, the bus last came `b|a` minutes before our arrival, thus we have to wait `b-b|a` minutes for it to arrive again.

In [34]:
({⊃buses[⊃⍋⍵]}×{⊃⍵[⊢⍋⍵]})buses-buses|arrival

2215

Problem two looks a lot like chinese remainder theorem. For e.g. `(id offset)` pairs `(19 0) (41 9)` we look for a solution `x` to:
$$
    x ≡ 0 \, (\operatorname{mod} 19) \\
    x ≡ 9 \, (\operatorname{mod} 41)
$$

In [397]:
]dinput
chinese←{ ⍝ ⍺ the residues, ⍵ the moduli
    d y _ ← ⍵[1] egcd ⍵[2]
    na nw ←, (⍺[1]-y×⍵[1]×(⍺[1]-⍺[2])÷d) ((⍵[1]×⍵[2])÷d)
    ⍝ If there were only 2 congruences left, we are done
    2=≢⍺: nw na
    ⍝ Otherwise, recurse
    (na, 2↓⍺)∇(nw, 2↓⍵)
}

In [408]:
⍝ Extended Euclidean Algorithm for ⍺ and ⍵.
egcd←{⍺=0: ⍵ 0 1 ⋄ d x y←(⍺|⍵)∇⍺ ⋄ d (y-x×⌊⍵÷⍺) x}
⍝ This would be the solution; it works for the test inputs,
⍝ so it’s probably algorithmically correct, but the real input
⍝ seems to be way too big and I don’t get APL to work with bigints.
⍝ -/|\(xbuses[1;] chinese xbuses[2;])

⍝ This is the answer as produced with an online CRT calculator.
-/|\(∧/xbuses[2;]) 490590248452237

1058443396696792

## Day 14

In [424]:
input14 ← ⊃⎕NGET 'day14.txt' 1
ints ← {⍎¨⍵(∊⊆⊣)⎕D}
parseProg ← {'mask'(⊣≡∩)⍵: ⊃¯1↑' '(≠⊆⊢)⍵ ⋄ ints ⍵}
parsed14 ← parseProg¨ input14

In [65]:
]dinput
appMasks←{ ⍝ ⍺ the program, ⍵ the current mask; result is the updated mem-commands
    0=≢⍺: ⍬
    cmd←⊃⍺ ⍝ Read the next command
    2≠≢cmd: (1↓⍺) ∇ cmd ⍝ mask command
    addr val←cmd ⍝ Destructure mem command
    (⊂addr, val appMask ⍵) ,((1↓⍺) ∇ ⍵)
}

In [79]:
⎕PP←34
⍝ ⍺ the value, ⍵ the mask
appMask←{2⊥⍎¨⊃¨(⍕¨('X'⍷⍵)/(36⍴2)⊤⍺)@{'X'⍷⍵}⍵}
+⌿(⊃¨{¯1↑⍵}⌸2∘⊃¨)(parsed14) appMasks 36⍴'X'

14839536808842

For problem two, we have to mask the addresses, but in a different way (I hope that the operators are correct, but I will see that):
* `mask0 ← mask`, but X’s replaced by 0’s
* `masks ← vector of mask’s`, but replace ~X’s by 1 and cycle through all possible X’s 
* `address ∨¨ mask0` is the old address, but every 1 in the mask replaces the value in address
* `(address ∨¨ mask0) ∘.∧¨ masks` is the vector of all addresses that are to be written to

In [416]:
]dinput
appMask2←{ ⍝ ⍺ the address, ⍵ the mask; return all addresses
    cntX←+/'X'⍷⍵
    size←2*cntX
    ⍝ Replace all X’s by 1’s
    mask0←'1'@{'X'⍷⍵} ⍵
    ⍝ Replace all not-X’s by 1’s:
    masks←'1'@{~'X'⍷⍵}⍵
    ⍝ Cycle through all X’s:
    masks←↓(size 36)⍴(∊,⌿⍕¨(cntX⍴2)⊤⍳(2*cntX))@{'X'⍷⍵}∊(2*cntX)⍴⊂masks
    ⍝ newAddr = (addr ∨ mask[X=1]) ∧ changedMask[~X=1]
    2⊥¨(⍎¨¨masks) ∧¨ ⊂((36⍴2)⊤⍺) ∨ ⍎¨mask0
}

In [324]:
]dinput
appMasks2←{ ⍝ ⍺ the program, ⍵ the current mask; result is all mem-commands
    0=≢⍺: ⍬
    cmd←⊃⍺ ⍝ Read the next command
    2≠≢cmd: (1↓⍺) ∇ cmd ⍝ mask command
    addr val←cmd ⍝ Destructure mem command
    (⊂(addr appMask2 ⍵),¨ val) ,((1↓⍺) ∇ ⍵)
}

In [433]:
+⌿((∊⊃¨¨){¯1↑⍵}⌸(∊2∘⊃¨¨))(parsed14) appMasks2 36⍴'X'

4215284199669

## Day 15

In [465]:
⊢input15← ⍎¨',' (≠⊆⊢) ⊃⊃⎕NGET 'day15.txt' 1

6 19 0 5 7 13 1

In [462]:
]dinput
day15_1←{ ⍝ ⍺ the number of to-count numbers, ⍵ the numbers up to now (with the first one being the most recent)
    ⍺=≢⍵: ⍵ ⍝ No more numbers → the most recent number is the solution
    new←(≢⍵)|(1↓⍵)⍳⊃⍵
    ⍺∇(new,⍵)
}

In [453]:
{(≢⍵)|(1↓⍵)⍳⊃⍵}0,⌽input15

5

In [467]:
⊃2020 day15_1 ⌽input15

468

In [ ]:
30000000 day15_1 ⌽ input15